# Disabling Windows Event Auditing
Detects scenarios where system auditing (ie: windows event log auditing) is disabled. This may be used in a scenario where an entity would want to bypass local logging to evade detection when windows event logging is enabled and reviewed. Also, it is recommended to turn off "Local Group Policy Object Processing" via GPO, which will make sure that Active Directory GPOs take precedence over local/edited computer policies via something such as "gpedit.msc". Please note, that disabling "Local Group Policy Object Processing" may cause an issue in scenarios of one off specific GPO modifications -- however it is recommended to perform these modifications in Active Directory anyways.

## Rule Content
```
- title: Disabling Windows Event Auditing
  id: 69aeb277-f15f-4d2d-b32a-55e883609563
  description: 'Detects scenarios where system auditing (ie: windows event log auditing)
    is disabled. This may be used in a scenario where an entity would want to bypass
    local logging to evade detection when windows event logging is enabled and reviewed.
    Also, it is recommended to turn off "Local Group Policy Object Processing" via
    GPO, which will make sure that Active Directory GPOs take precedence over local/edited
    computer policies via something such as "gpedit.msc". Please note, that disabling
    "Local Group Policy Object Processing" may cause an issue in scenarios of one
    off specific GPO modifications -- however it is recommended to perform these modifications
    in Active Directory anyways.'
  references:
  - https://bit.ly/WinLogsZero2Hero
  tags:
  - attack.defense_evasion
  - attack.t1054
  author: '@neu5ron'
  logsource:
    product: windows
    service: security
    definition: 'Requirements: Audit Policy : Computer Management > Audit Policy Configuration,
      Group Policy : Computer Configuration\Windows Settings\Security Settings\Advanced
      Audit Policy Configuration\Audit Policies\Policy Change\Audit Authorization
      Policy Change'
    category: null
  detection:
    selection:
      EventID: 4719
      AuditPolicyChanges: removed
    condition: selection
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4719" AND policy_changes:"removed")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()